In [1]:
#| default_exp auto

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export
from os import cpu_count
import numpy as np
import torch

from ray import tune
from ray.tune.search.basic_variant import BasicVariantGenerator

from neuralforecast.common._base_auto import BaseAuto

from neuralforecast.models.rnn import RNN
from neuralforecast.models.gru import GRU
from neuralforecast.models.tcn import TCN
from neuralforecast.models.lstm import LSTM
from neuralforecast.models.dilated_rnn import DilatedRNN

from neuralforecast.models.mlp import MLP
from neuralforecast.models.nbeats import NBEATS
from neuralforecast.models.nbeatsx import NBEATSx
from neuralforecast.models.nhits import NHITS

from neuralforecast.models.tft import TFT
from neuralforecast.models.vanillatransformer import VanillaTransformer
from neuralforecast.models.informer import Informer
from neuralforecast.models.autoformer import Autoformer
from neuralforecast.models.patchtst import PatchTST

from neuralforecast.models.stemgnn import StemGNN
from neuralforecast.models.hint import HINT

from neuralforecast.losses.pytorch import MAE

In [4]:
#| hide
import matplotlib.pyplot as plt

from fastcore.test import test_eq
from nbdev.showdoc import show_doc

import logging
import warnings

from neuralforecast.losses.pytorch import MSE

In [5]:
#| hide
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

plt.rcParams["axes.grid"]=True
plt.rcParams['font.family'] = 'serif'
plt.rcParams["figure.figsize"] = (6,4)

# <span style="color:DarkOrange"> Models </span>

> NeuralForecast contains user-friendly implementations of neural forecasting models that allow for easy transition of computing capabilities (GPU/CPU), computation parallelization, and hyperparameter tuning.<br><br> All the NeuralForecast models are "global" because we train them with all the series from the input pd.DataFrame data `Y_df`, yet the optimization objective is, momentarily, "univariate" as it does not consider the interaction between the output predictions across time series. Like the StatsForecast library, `core.NeuralForecast` allows you to explore collections of models efficiently and contains functions for convenient wrangling of input and output pd.DataFrames predictions.

# <span style="color:DarkBlue"> 1. Automatic Forecasting </span>

## <span style="color:DarkBlue"> A. RNN-Based </span>

In [6]:
#| export
class AutoRNN(BaseAuto):
    
    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):
        """ Auto RNN
        
        **Parameters:**<br>
        
        """
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            config['inference_input_size'] = tune.choice([h*x \
                         for x in self.default_config["inference_input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoRNN, self).__init__(
              cls_model=RNN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [7]:
show_doc(AutoRNN, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L41){target="_blank" style="float:right; font-size:smaller"}

### AutoRNN

>      AutoRNN (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x7f994033c6a0>, num_samples=10, refit_with_val=False,
>               cpus=4, gpus=0, verbose=False)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [8]:
%%capture
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengersDF as Y_df

# Split train/test and declare time series dataset
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31']   # 12 test
dataset, *_ = TimeSeriesDataset.from_df(Y_train_df)

In [9]:
%%capture
# Use your own config or AutoRNN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoRNN(h=12, config=config, num_samples=1, cpus=1)

model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [10]:
# Plotting predictions
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoRNN'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [11]:
#| export
class AutoLSTM(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            config['inference_input_size'] = tune.choice([h*x \
                         for x in self.default_config["inference_input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoLSTM, self).__init__(
              cls_model=LSTM,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose
        )

In [12]:
show_doc(AutoLSTM, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L101){target="_blank" style="float:right; font-size:smaller"}

### AutoLSTM

>      AutoLSTM (h, loss=MAE(), valid_loss=None, config=None,
>                search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>                object at 0x7f994142c6d0>, num_samples=10,
>                refit_with_val=False, cpus=4, gpus=0, verbose=False)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [13]:
%%capture
# Use your own config or AutoLSTM.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoLSTM(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [14]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoLSTM'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [15]:
#| export
class AutoGRU(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "encoder_n_layers": tune.randint(1, 4),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            config['inference_input_size'] = tune.choice([h*x \
                         for x in self.default_config["inference_input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoGRU, self).__init__(
              cls_model=GRU,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [16]:
show_doc(AutoGRU, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L156){target="_blank" style="float:right; font-size:smaller"}

### AutoGRU

>      AutoGRU (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x7f994148d2b0>, num_samples=10, refit_with_val=False,
>               cpus=4, gpus=0, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [17]:
%%capture
# Use your own config or AutoGRU.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoGRU(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [18]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoGRU'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [19]:
#| export
class AutoTCN(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            config['inference_input_size'] = tune.choice([h*x \
                         for x in self.default_config["inference_input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoTCN, self).__init__(
              cls_model=TCN,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [20]:
show_doc(AutoTCN, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L213){target="_blank" style="float:right; font-size:smaller"}

### AutoTCN

>      AutoTCN (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x7f994103cc40>, num_samples=10, refit_with_val=False,
>               cpus=4, gpus=0, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [21]:
%%capture
# Use your own config or AutoTCN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoTCN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [22]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoTCN'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [23]:
#| export
class AutoDilatedRNN(BaseAuto):

    default_config = {
        "input_size_multiplier": [-1, 4, 16, 64],
        "inference_input_size_multiplier": [-1],
        "h": None,
        "cell_type": tune.choice(['LSTM', 'GRU']),
        "encoder_hidden_size": tune.choice([50, 100, 200, 300]),
        "dilations": tune.choice([ [[1, 2], [4, 8]], [[1, 2, 4, 8]] ]),
        "context_size": tune.choice([5, 10, 50]),
        "decoder_hidden_size": tune.choice([64, 128, 256, 512]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([16, 32]),
        "loss": None,
        "random_seed": tune.randint(1, 20)
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            config['inference_input_size'] = tune.choice([h*x \
                         for x in self.default_config["inference_input_size_multiplier"]])
            del config["input_size_multiplier"]

        super(AutoDilatedRNN, self).__init__(
              cls_model=DilatedRNN,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
         )

In [24]:
show_doc(AutoDilatedRNN, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L269){target="_blank" style="float:right; font-size:smaller"}

### AutoDilatedRNN

>      AutoDilatedRNN (h, loss=MAE(), valid_loss=None, config=None,
>                      search_alg=<ray.tune.search.basic_variant.BasicVariantGen
>                      erator object at 0x7f9942d4cd30>, num_samples=10,
>                      refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                      alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [25]:
%%capture
# Use your own config or AutoDilatedRNN.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=-1, encoder_hidden_size=8)
model = AutoDilatedRNN(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [26]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoDilatedRNN'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

## <span style="color:DarkBlue"> B. MLP-Based </span>

In [27]:
#| export
class AutoMLP(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice( [256, 512, 1024] ),
        "num_layers": tune.randint(2, 6),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,     
                 config=None,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoMLP, self).__init__(
              cls_model=MLP,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config, 
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [28]:
show_doc(AutoMLP, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L327){target="_blank" style="float:right; font-size:smaller"}

### AutoMLP

>      AutoMLP (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x7f9942d518b0>, num_samples=10, refit_with_val=False,
>               cpus=4, gpus=0, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [29]:
%%capture
# Use your own config or AutoMLP.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoMLP(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [30]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoMLP'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [31]:
#| export
class AutoNBEATS(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoNBEATS, self).__init__(
              cls_model=NBEATS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [32]:
show_doc(AutoNBEATS, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L384){target="_blank" style="float:right; font-size:smaller"}

### AutoNBEATS

>      AutoNBEATS (h, loss=MAE(), valid_loss=None, config=None,
>                  search_alg=<ray.tune.search.basic_variant.BasicVariantGenerat
>                  or object at 0x7f9942d967f0>, num_samples=10,
>                  refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                  alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [33]:
%%capture
# Use your own config or AutoNBEATS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12,
              mlp_units=3*[[8, 8]])
model = AutoNBEATS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [34]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoNBEATS'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [35]:
#| export
class AutoNBEATSx(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoNBEATSx, self).__init__(
              cls_model=NBEATSx,
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [36]:
show_doc(AutoNBEATSx, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L439){target="_blank" style="float:right; font-size:smaller"}

### AutoNBEATSx

>      AutoNBEATSx (h, loss=MAE(), valid_loss=None, config=None,
>                   search_alg=<ray.tune.search.basic_variant.BasicVariantGenera
>                   tor object at 0x7f994148dca0>, num_samples=10,
>                   refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                   alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [37]:
%%capture
# Use your own config or AutoNBEATS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12,
              mlp_units=3*[[8, 8]])
model = AutoNBEATSx(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [38]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoNBEATSx'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [39]:
#| export
class AutoNHITS(BaseAuto):

    default_config = {
       "input_size_multiplier": [1, 2, 3, 4, 5],
       "h": None,
       "n_pool_kernel_size": tune.choice([[2, 2, 1], 3*[1], 3*[2], 3*[4], 
                                         [8, 4, 1], [16, 8, 1]]),
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], 
                                         [180, 60, 1], [60, 8, 1], 
                                         [40, 20, 1], [1, 1, 1]]),
       "learning_rate": tune.loguniform(1e-4, 1e-1),
       "scaler_type": tune.choice([None, 'robust', 'standard']),
       "max_steps": tune.quniform(lower=500, upper=1500, q=100),
       "batch_size": tune.qloguniform(lower=5, upper=9, base=2, q=1), #[32, 64, 128, 256]
       "windows_batch_size": tune.qloguniform(lower=7, upper=10, base=2, q=1), #[128, 256, 512, 1024]
       "loss": None,
       "random_seed": tune.randint(lower=1, upper=20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):

        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])
            
            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoNHITS, self).__init__(
              cls_model=NHITS, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples,
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [40]:
show_doc(AutoNHITS, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L494){target="_blank" style="float:right; font-size:smaller"}

### AutoNHITS

>      AutoNHITS (h, loss=MAE(), valid_loss=None, config=None,
>                 search_alg=<ray.tune.search.basic_variant.BasicVariantGenerato
>                 r object at 0x7f994269d340>, num_samples=10,
>                 refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                 alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [41]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=2, val_check_steps=1, input_size=12, 
              mlp_units=3 * [[8, 8]])
model = AutoNHITS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [42]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoNHITS'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

## <span style="color:DarkBlue"> C. Transformer-Based </span>

In [43]:
#| export
class AutoTFT(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoTFT, self).__init__(
              cls_model=TFT, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [44]:
show_doc(AutoTFT, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L564){target="_blank" style="float:right; font-size:smaller"}

### AutoTFT

>      AutoTFT (h, loss=MAE(), valid_loss=None, config=None,
>               search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>               object at 0x7f9942706d60>, num_samples=10, refit_with_val=False,
>               cpus=4, gpus=0, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [45]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoTFT(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [46]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoTFT'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [47]:
#| export
class AutoVanillaTransformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoVanillaTransformer, self).__init__(
              cls_model=VanillaTransformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [48]:
show_doc(AutoVanillaTransformer, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L621){target="_blank" style="float:right; font-size:smaller"}

### AutoVanillaTransformer

>      AutoVanillaTransformer (h, loss=MAE(), valid_loss=None, config=None,
>                              search_alg=<ray.tune.search.basic_variant.BasicVa
>                              riantGenerator object at 0x7f9942e7d700>,
>                              num_samples=10, refit_with_val=False, cpus=4,
>                              gpus=0, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [49]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoVanillaTransformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [50]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoVanillaTransformer'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [51]:
#| export
class AutoInformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoInformer, self).__init__(
              cls_model=Informer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [52]:
show_doc(AutoInformer, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L678){target="_blank" style="float:right; font-size:smaller"}

### AutoInformer

>      AutoInformer (h, loss=MAE(), valid_loss=None, config=None,
>                    search_alg=<ray.tune.search.basic_variant.BasicVariantGener
>                    ator object at 0x7f9942e9aca0>, num_samples=10,
>                    refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                    alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [53]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoInformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [54]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoInformer'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [55]:
#| export
class AutoAutoformer(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4, 5],
        "h": None,
        "hidden_size": tune.choice([64, 128, 256]),
        "n_head": tune.choice([4, 8]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoAutoformer, self).__init__(
              cls_model=Autoformer, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [56]:
show_doc(AutoAutoformer, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L735){target="_blank" style="float:right; font-size:smaller"}

### AutoAutoformer

>      AutoAutoformer (h, loss=MAE(), valid_loss=None, config=None,
>                      search_alg=<ray.tune.search.basic_variant.BasicVariantGen
>                      erator object at 0x7f9943113790>, num_samples=10,
>                      refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                      alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [57]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=8)
model = AutoAutoformer(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [58]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoAutoformer'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [59]:
#| export
class AutoPatchTST(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3],
        "h": None,
        "hidden_size": tune.choice([16, 128, 256]),
        "n_head": tune.choice([4, 16]),
        "patch_len": tune.choice([16, 24]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "revin": tune.choice([False, True]),
        "max_steps": tune.choice([500, 1000, 5000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "windows_batch_size": tune.choice([128, 256, 512, 1024]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        super(AutoPatchTST, self).__init__(
              cls_model=PatchTST, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [60]:
show_doc(AutoPatchTST, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L792){target="_blank" style="float:right; font-size:smaller"}

### AutoPatchTST

>      AutoPatchTST (h, loss=MAE(), valid_loss=None, config=None,
>                    search_alg=<ray.tune.search.basic_variant.BasicVariantGener
>                    ator object at 0x7f9943119070>, num_samples=10,
>                    refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                    alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [61]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12, hidden_size=16)
model = AutoPatchTST(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [62]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoPatchTST'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

## <span style="color:DarkBlue"> D. Multivariate </span>

In [63]:
#| export
class AutoStemGNN(BaseAuto):

    default_config = {
        "input_size_multiplier": [1, 2, 3, 4],
        "h": None,
        "n_series": None,
        "n_stacks": tune.choice([2, 3]),
        "multi_layer": tune.choice([3, 5, 7]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        "scaler_type": tune.choice([None, 'robust', 'standard']),
        "max_steps": tune.choice([500, 1000, 2000]),
        "batch_size": tune.choice([32, 64, 128, 256]),
        "loss": None,
        "random_seed": tune.randint(1, 20),
    }

    def __init__(self,
                 h,
                 n_series,
                 loss=MAE(),
                 valid_loss=None,
                 config=None, 
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 refit_with_val=False,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 verbose=False,
                 alias=None):
        
        # Define search space, input/output sizes
        if config is None:
            config = self.default_config.copy()        
            config['input_size'] = tune.choice([h*x \
                         for x in self.default_config["input_size_multiplier"]])

            # Rolling windows with step_size=1 or step_size=h
            # See `BaseWindows` and `BaseRNN`'s create_windows
            config['step_size'] = tune.choice([1, h])
            del config["input_size_multiplier"]

        # Always use n_series from parameters
        config['n_series'] = n_series

        super(AutoStemGNN, self).__init__(
              cls_model=StemGNN, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )

In [64]:
show_doc(AutoStemGNN, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L851){target="_blank" style="float:right; font-size:smaller"}

### AutoStemGNN

>      AutoStemGNN (h, n_series, loss=MAE(), valid_loss=None, config=None,
>                   search_alg=<ray.tune.search.basic_variant.BasicVariantGenera
>                   tor object at 0x7f99430ee1f0>, num_samples=10,
>                   refit_with_val=False, cpus=4, gpus=0, verbose=False,
>                   alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [65]:
%%capture
# Use your own config or AutoNHITS.default_config
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoStemGNN(h=12, n_series=1, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

Global seed set to 1


In [66]:
#| hide
# Y_plot_df = Y_test_df[['unique_id', 'ds', 'y']].copy()
# Y_plot_df['AutoStemGNN'] = y_hat

# pd.concat([Y_train_df, Y_plot_df]).drop('unique_id', axis=1).set_index('ds').plot()

In [67]:
#| export
class AutoHINT(BaseAuto):

    def __init__(self,
                 cls_model,
                 h,
                 loss,
                 valid_loss,
                 S,
                 config,
                 search_alg=BasicVariantGenerator(random_state=1),
                 num_samples=10,
                 cpus=cpu_count(),
                 gpus=torch.cuda.device_count(),
                 refit_with_val=False,
                 verbose=False,
                 alias=None):
        super(AutoHINT, self).__init__(
              cls_model=cls_model, 
              h=h,
              loss=loss,
              valid_loss=valid_loss,
              config=config,
              search_alg=search_alg,
              num_samples=num_samples, 
              refit_with_val=refit_with_val,
              cpus=cpus,
              gpus=gpus,
              verbose=verbose,
              alias=alias,
        )
        # Validate presence of reconciliation strategy
        # parameter in configuration space
        if not ('reconciliation' in config.keys()):
            raise Exception("config needs reconciliation, \
                            try tune.choice(['BottomUp', 'MinTraceOLS', 'MinTraceWLS'])")
        self.S = S

    def _fit_model(self, cls_model, config,
                   dataset, val_size, test_size):
        # Overwrite _fit_model for HINT two-stage instantiation
        reconciliation = config.pop('reconciliation')
        base_model = cls_model(**config)
        model = HINT(h=base_model.h, model=base_model, 
                     S=self.S, reconciliation=reconciliation)
        model.fit(
            dataset,
            val_size=val_size, 
            test_size=test_size
        )
        return model

In [68]:
show_doc(AutoHINT, title_level=3)

---

[source](https://github.com/Nixtla/neuralforecast/blob/main/neuralforecast/auto.py#L912){target="_blank" style="float:right; font-size:smaller"}

### AutoHINT

>      AutoHINT (cls_model, h, loss, valid_loss, S, config,
>                search_alg=<ray.tune.search.basic_variant.BasicVariantGenerator
>                object at 0x7f99434f8670>, num_samples=10, cpus=4, gpus=0,
>                refit_with_val=False, verbose=False, alias=None)

Class for Automatic Hyperparameter Optimization, it builds on top of `ray` to
give access to a wide variety of hyperparameter optimization tools ranging
from classic grid search, to Bayesian optimization and HyperBand algorithm.

The validation loss to be optimized is defined by the `config['loss']` dictionary
value, the config also contains the rest of the hyperparameter search space.

It is important to note that the success of this hyperparameter optimization
heavily relies on a strong correlation between the validation and test periods.

**Parameters:**<br>
`cls_model`: PyTorch/PyTorchLightning model, see `neuralforecast.models` [collection here](https://nixtla.github.io/neuralforecast/models.html).<br>
`h`: int, forecast horizon.<br>
`loss`: PyTorch module, instantiated train loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`valid_loss`: PyTorch module=`loss`, instantiated valid loss class from [losses collection](https://nixtla.github.io/neuralforecast/losses.pytorch.html).<br>
`config`: dict, dictionary with ray.tune defined search space.<br>
`search_alg`: ray.tune.search variant, BasicVariantGenerator, HyperOptSearch, DragonflySearch, TuneBOHB for details
    see [tune.search](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#).<br>
`num_samples`: int, number of hyperparameter optimization steps/samples.<br>
`cpus`: int, number of cpus to use during optimization, default all available.<br>
`gpus`: int, number of gpus to use during optimization, default all available.<br>
`refit_wo_val`: bool, number of gpus to use during optimization, default all available.<br>
`verbose`: bool, wether print partial outputs.<br>
`alias`: str, optional,  Custom name of the model.<br>

In [69]:
#| hide
def sort_df_hier(Y_df, S_df):
    # NeuralForecast core, sorts unique_id lexicographically
    # by default, this class matches S_df and Y_hat_df order.    
    Y_df.unique_id = Y_df.unique_id.astype('category')
    Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
    Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
    return Y_df

# -----Create synthetic dataset-----
np.random.seed(123)
train_steps = 20
num_levels = 7
level = np.arange(0, 100, 0.1)
qs = [[50-lv/2, 50+lv/2] for lv in level]
quantiles = np.sort(np.concatenate(qs)/100)

levels = ['Top', 'Mid1', 'Mid2', 'Bottom1', 'Bottom2', 'Bottom3', 'Bottom4']
unique_ids = np.repeat(levels, train_steps)

S = np.array([[1., 1., 1., 1.],
              [1., 1., 0., 0.],
              [0., 0., 1., 1.],
              [1., 0., 0., 0.],
              [0., 1., 0., 0.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]])

S_dict = {col: S[:, i] for i, col in enumerate(levels[3:])}
S_df = pd.DataFrame(S_dict, index=levels)

ds = pd.date_range(start='2018-03-31', periods=train_steps, freq='Q').tolist() * num_levels
# Create Y_df
y_lists = [S @ np.random.uniform(low=100, high=500, size=4) for i in range(train_steps)]
y = [elem for tup in zip(*y_lists) for elem in tup]
Y_df = pd.DataFrame({'unique_id': unique_ids, 'ds': ds, 'y': y})
Y_df = sort_df_hier(Y_df, S_df)

hint_dataset, *_ = TimeSeriesDataset.from_df(df=Y_df)

In [70]:
%%capture
# Perform a simple hyperparameter optimization with 
# NHITS and then reconcile with HINT
from neuralforecast.losses.pytorch import GMM, sCRPS

base_config = dict(max_steps=1, val_check_steps=1, input_size=8)
base_model = AutoNHITS(h=4, loss=GMM(n_components=2, quantiles=quantiles), 
                       config=base_config, num_samples=1, cpus=1)
model = HINT(h=4, S=S_df.values,
             model=base_model,  reconciliation='MinTraceOLS')

model.fit(dataset=dataset)
y_hat = model.predict(dataset=hint_dataset)

# Perform a conjunct hyperparameter optimization with 
# NHITS + HINT reconciliation configurations
nhits_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "windows_batch_size": tune.choice([256]),                                 # Number of windows in batch
       "n_pool_kernel_size": tune.choice([[2, 2, 2], [16, 8, 1]]),               # MaxPool's Kernelsize
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], [1, 1, 1]]), # Interpolation expressivity ratios
       "activation": tune.choice(['ReLU']),                                      # Type of non-linear activation
       "n_blocks":  tune.choice([[1, 1, 1]]),                                    # Blocks per each 3 stacks
       "mlp_units":  tune.choice([[[512, 512], [512, 512], [512, 512]]]),        # 2 512-Layers per block for each stack
       "interpolation_mode": tune.choice(['linear']),                            # Type of multi-step interpolation
       "random_seed": tune.randint(1, 10),
       "reconciliation": tune.choice(['BottomUp', 'MinTraceOLS', 'MinTraceWLS'])
    }
model = AutoHINT(h=4, S=S_df.values,
                 cls_model=NHITS,
                 config=nhits_config,
                 loss=GMM(n_components=2, level=[80, 90]),
                 valid_loss=sCRPS(level=[80, 90]),
                 num_samples=1, cpus=1)
model.fit(dataset=dataset)
y_hat = model.predict(dataset=hint_dataset)

Global seed set to 1
Global seed set to 6


# TESTS

In [71]:
#| hide
## TESTS
config = dict(max_steps=1, val_check_steps=1, input_size=12)
model = AutoNHITS(h=12, config=config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

(_train_tune pid=18044) Global seed set to 1


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s]                             


Global seed set to 1
(_train_tune pid=18044) /opt/anaconda3/envs/neuralforecast/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(_train_tune pid=18044)   warnings.warn('resource_tracker: There appear to be %d '


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s, v_num=0, train_loss_step=39.50]
Validation: 0it [00:00, ?it/s] 
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 127.78it/s]


In [72]:
#| hide
## TESTS
nhits_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "windows_batch_size": tune.choice([256]),                                 # Number of windows in batch
       "n_pool_kernel_size": tune.choice([[2, 2, 2], [16, 8, 1]]),               # MaxPool's Kernelsize
       "n_freq_downsample": tune.choice([[168, 24, 1], [24, 12, 1], [1, 1, 1]]), # Interpolation expressivity ratios
       "activation": tune.choice(['ReLU']),                                      # Type of non-linear activation
       "n_blocks":  tune.choice([[1, 1, 1]]),                                    # Blocks per each 3 stacks
       "mlp_units":  tune.choice([[[512, 512], [512, 512], [512, 512]]]),        # 2 512-Layers per block for each stack
       "interpolation_mode": tune.choice(['linear']),                            # Type of multi-step interpolation
       "random_seed": tune.randint(1, 10),
    }

model = AutoNHITS(h=12, loss=MAE(), valid_loss=MSE(), config=nhits_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

# Test equality
test_eq(str(type(model.valid_loss)), "<class 'neuralforecast.losses.pytorch.MSE'>")

(_train_tune pid=18056) Global seed set to 2


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]


Global seed set to 2


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 10.01it/s, v_num=0, train_loss_step=52.60]
Validation: 0it [00:00, ?it/s] 
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  8.38it/s, v_num=0, train_loss_step=52.60, valid_loss=3.45e+4, train_loss_epoch=52.60]
                                                                           

(_train_tune pid=18056) /opt/anaconda3/envs/neuralforecast/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(_train_tune pid=18056)   warnings.warn('resource_tracker: There appear to be %d '


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 156.60it/s]


In [73]:
#| hide
## TODO: Add unit tests for interactions between loss/valid_loss types
## TODO: Unit tests (2 types of networks x 2 types of loss x 2 types of valid loss)
from neuralforecast.losses.pytorch import GMM, sCRPS

## Checking if base recurrent methods run point valid_loss correctly
tcn_config = {
       "learning_rate": tune.choice([1e-3]),                                     # Initial Learning rate
       "max_steps": tune.choice([1]),                                            # Number of SGD steps
       "val_check_steps": tune.choice([1]),                                      # Number of steps between validation
       "input_size": tune.choice([5 * 12]),                                      # input_size = multiplier * horizon
       "batch_size": tune.choice([7]),                                           # Number of series in windows
       "random_seed": tune.randint(1, 10),
    }

model = AutoTCN(h=12, 
                loss=MAE(), 
                valid_loss=MSE(), 
                config=tcn_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

## Checking if base recurrent methods run quantile valid_loss correctly
model = AutoTCN(h=12, 
                loss=GMM(n_components=2, level=[80, 90]),
                valid_loss=sCRPS(level=[80, 90]),
                config=tcn_config, num_samples=1, cpus=1)

# Fit and predict
model.fit(dataset=dataset)
y_hat = model.predict(dataset=dataset)

(_train_tune pid=18074) Global seed set to 9


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]


Global seed set to 9
(_train_tune pid=18074) /opt/anaconda3/envs/neuralforecast/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(_train_tune pid=18074)   warnings.warn('resource_tracker: There appear to be %d '


Epoch 0: 100%|██████████| 1/1 [00:00<00:00, 56.02it/s, v_num=0, train_loss_step=5.510]
Validation: 0it [00:00, ?it/s] 
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
(_train_tune pid=18074) 
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  7.55it/s]
